# Laboratorio 7: Clasificación 🤗

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>


### Cuerpo Docente:

- Profesores: Ignacio Meza, Sebastián Tinoco
- Auxiliares: Catherine Benavides y Consuelo Rojas
- Ayudante: Nicolás Ojeda, Eduardo Moya

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Vanessa González
- Nombre de alumno 2: Benjamín Angulo

### Temas a tratar
- Clasificación en problemas desbalanceados
- Lightgbm y xgboost
- Pipelines

### Reglas:

- **Grupos de 2 personas**
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias.
- Pueden usar cualquer matrial del curso que estimen conveniente.
- Código que no se pueda ejecutar, no será revisado.



### Objetivos principales del laboratorio
- Comprender cómo trabajar con problemas de clasificación con clases desbalanceadas.
- Aplicar los modelos lightgbm y xgboost.
- Practicar Pipelines

# Parte Teórica [12 puntos]




1. Explique cuál es la diferencia entre los datos de entrenamiento y validación. [1 punto]

2. Explique cuál es el principal desafío al trabajar problemas de clasificación con data no supervisada. [1 punto]

3. Explique en **sus palabras** qué es la matriz de confusión y para qué se utiliza. [1 puntos]

4. Escriba la fórmula de las siguientes métricas y explique con **sus palabras** cómo se interpretan. [1 punto cada uno]

  * Accuracy
  * Precision
  * Recall
  * F1 score

5. Explique qué métrica recomendaría para los siguientes contextos de clasificación. [1 punto cada uno]

  * Mantenimiento predictivo de fallas de maquinaria pesada en la industria minera.  
  * Detección de enfermedades altamente contagiosas.
  * Aprobación de créditos de alto riesgo.
  * Detección de crímenes.

6. Explique qué es la calibración de modelos y para qué se usa. [1 punto]


**Respuesta:**

1.- La diferencia entre estos radica en que los datos de entrenamiento son datos tomados a partir de una partición de un dataset original, siendo este conjunto el más grande en comparación al de validación, ya que se usará para entrenar el modelo de Machine Learning. En cambio los datos de validación son utilizados para poder comprobar si el modelo está funcionando correctamente, ya que con estos se realizarán predicciones y se comprobará que los resultados predichos sean coherentes con los resultados reales.

2.- Los desafíos más importantes para este tipo de problemas son las evaluaciones de rendimiento, ya que al no haber clases definidas formalmente al momento de entrenar no se pueden usar las métricas que se usan en el entrenamiento con datos supervisados como podrían ser el accuracy, precision o recall.

3.- La matriz de confusión es una herramienta que permite visualizar las etiquetas predichas por un modelo de aprendizaje automático vs las etiquetas reales de la data. De este modo se puede tener una mejor perspectiva de como funciona el modelo al ver su funcionamiento con la data de validación sin necesariamente usar las métricas clásicas como puede ser el recall o precision.

4.- **Accuracy**:   $accuracy = \frac{TP + TN}{TP + TN + FP + FN}$   El accuracy entrega la proporción de datos predecidos correctamente con respecto al total de los datos.

**Precision**:   $precision = \frac{TP}{TP + FP}$   La precision entrega la proporción de datos con etiqueta positiva predecidos correctamente de todos los datos predecidos como positivos.

**Recall**:    $recall = \frac{TP}{TP + FN}$    El recall entrega la proporción de datos con etiqueta positiva predichos correctamente de todos los datos con etiqueta positiva real.

**F1 score**:    $F1 score = \frac{2 \cdot precision \cdot recall}{precision + recall}$     El F1 score mide el balance entre precision y recall, por lo que un valor alto de este implica una buena predicción del modelo de las clases positivas.

5.-

**Mantenimiento predictivo de fallas de maquinaria pesada en la industria minera**: Cómo se espera poder predecir de manera correcta los mantenimientos la mejor métrica sería le recall, ya que lo más perjudicial para este modelo son los falsos negativos, ya que implicaría que la maquinaría podría fallar antes del mantenimiento. Así mismo, los falsos positivos podrían significar un problema por gastos innecesarios en mantención, pero el peso que tiene esto es menor al que tiene los falsos negativos. De todos modos la precision sería una métrica importante aunque menos que el recall.

**Detección de enfermedades altamente contagiosas**: Para este caso es importante poder predecir todos los casos positivos, por lo que la prioridad sería poder tener la menor cantidad de falsos negativos para así poder evitar posibles contagios futuros. De este modo la métrica que tendría que tener un mejor valor para este modelo es el recall para así tener menos falsos negativos.

**Aprobación de créditos de alto riesgo**: Para este caso el principal peligro es el aprobar créditos que no debiesen ser aprobados. De este modo lo que se busca es un modelo con la menor cantidad de falsos positivos, por lo que la métrica más importante para este caso es la precision.

**Detección de crímenes**: En este caso es importante detectar la mayor cantidad de crímenes reales, por lo que el mayor interés es disminuir la cantidad de falsos negativos. De este modo la métrica más importante para este modelo sería el recall.

6.- La calibración consiste en el reajuste de probabilidades predichas para que correspondan con la proporción de casos reales observados. De este modo se busca realizar correcciones para ciertas predicciones cuando un modelo las subestima o las sobrestima. La calibración se usa para cuando se quiere emplear las probabilidades asociadas a la predicción en algún aspecto. Más allá de esto la calibración no tiene mucho valor, ya que no aporta a mejorar las predicciones de un modelo, sino que al contrario puede llegar a reducir el accuracy del mismo.
*Escriba su respuesta aquí*

# Parte práctica [48 puntos]

<p align="center">
  <img src="https://drive.google.com/uc?export=view&id=1BnO4tyh3vM2P199Ec9s3JjngQ4qQ9seP"
" width="300">
</p>


Tras el trágico despido de la mítica mascota de Maipú, Renacín decide adentrarse como consultor en el mercado futbolero, el cuál (para variar...) está cargado en especulaciones.

Como su principal tarea será asesorar a los directivos de los clubes sobre cuál jugador comprar y cuál no, Renacín desea generar modelos predictivos que evaluén distintas características de los jugadores; todo con el fin de tomar decisiones concretas basadas en los datos.

Sin embargo, su condición de corporeo le impidió tomar la versión anterior de MDS7202, por lo que este motivo Renacín contrata a su equipo para lograr su objetivo final. Dado que aún tiene fuertes vínculos con la dirección de deportes de la municipalidad, el corporeo le entrega base de datos con las estadísticas de cada jugador para que su equipo empieze a trabajar ya con un dataset listo para ser usado.


**Los Datos**

Para este laboratorio deberán trabajar con el csv `statsplayers.csv`, donde deberán aplicar algoritmos de aprendizaje supervisado de clasificación en base a características que describen de jugadores de fútbol.

Para comenzar cargue el dataset señalado y a continuación vea el reporte **`Player_Stats_Report.html`** (adjunto en la carpeta del enunciado) que describe las características principales del `DataFrame`.

In [2]:
# Si usted está utilizando Colabolatory le puede ser útil este código para cargar los archivos.
try:
    from google.colab import drive
    drive.mount("/content/drive")
    path = 'Dirección donde tiene los archivos en el Drive'
except:
    print('Ignorando conexión drive-colab')

Ignorando conexión drive-colab


In [3]:
import pandas as pd

df = pd.read_csv("stats_players.csv")

## 1. Predicción de Seleccionados Nacionales [14 puntos]

<p align="center">
  <img src="https://www.futuro.cl/wp-content/uploads/2016/06/chile-argentina-meme-12.jpg" width="300">
</p>



### 1.1 Preprocesamiento [5 puntos]

Tareas:

1. Genere los labels para la clasificación binaria en una variable llamada `label`. Para esto, trabaje sobre el atributo `National_Position` suponiendo que los valores nulos son jugadores no seleccionados para representar a su país. [Sin puntaje]

2. Hecho esto, ¿cuántos se tienen ejemplos por cada clase? Comente lo que observa. [1 punto]

3. Genere un `ColumnTransformer` en donde especifique las transformaciones que hay que realizar para cada columna (por ejemplo StandarScaler, MinMaxScaler, OneHotEncoder, etc...) para que puedan ser utilizadas correctamente por el modelo predictivo y guárdelo una variable llamada `col_transformer`. [2 puntos]

4. Comente y justifique las transformaciones elegidas sobre cada una de las variables (para esto utilice el material `Player_Stats_Report.html` que viene en el zip del lab), al igual que las transformaciones aplicadas. [2 puntos]

**Respuesta:**

In [4]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler

df["label"] = df["National_Position"].notnull().astype(int) # Se crea el atributo binario "label"

class_count = df["label"].value_counts() # Se hace un conteo de la cantidad de datos por clase del atributo "label"
print(class_count)

categorical_data = ["Club_Position", "Preffered_Foot", "Work_Rate"] # Se crea una lista con las variables categoricas, eliminando las que no aportan información como el nombre, nacionalidad y national position.
numerical_data_standard = ["Height", "Weight", "Age", "Ball_Control", "Dribbling", "Marking", "Sliding_Tackle", "Standing_Tackle", "Aggression", "Reactions", "Interceptions", "Vision", "Composure", "Crossing", "Short_Pass", "Long_Pass", "Acceleration", "Speed", "Stamina", "Strength", "Balance", "Agility", "Jumping", "Heading", "Shot_Power", "Finishing", "Long_Shots", "Curve", "Freekick_Accuracy", "Penalties", "Volleys"] # Se crea una lista con las variables numéricas con las que se usará la transformación StandardScaler
numerical_data_minmax = ["Weak_foot", "Skill_Moves"] # Se crea una lista con las variables numéricas con las que se usará la transformación MinMaxScaler

col_transformer = ColumnTransformer(
    transformers=[("categorical", OneHotEncoder(), categorical_data),
                  ("numerical_standard", StandardScaler(), numerical_data_standard),
                  ("numerical_minmax", MinMaxScaler(), numerical_data_minmax)]) # Se crea el ColumnTransformer con las transformaciones esperadas para cada conjunto de variables

label
0    16513
1     1075
Name: count, dtype: int64


2.- Se puede notar una gran disparidad entre ambas clases, lo que puede generar problemas a futuro en las predicciones de estas mismas que se buscan realizar.

4.- Se decide usar OneHotEncoder para las variables categóricas debido a que este permite convertir estas variables en arrays numéricos que podrán ser entendidos por los modelos. Además se decide usar StandardScaler para todas las variables numéricas que tienen una distribución gaussiana, con el fin de normalizarlas y así tener valores más concretos para los modelos. Por último en el caso de las variables numéricas "Weak_foot" y "Skill_Moves" se decide usar MinMaxScaler dada su distribución, ya que es una buena forma de acotar los valores de estos a [0,1], ya que no hay presencia de outliers y así se simplifica el procesamiento para el modelo.

### 1.2 Entrenamiento [3 puntos]

Ahora, vamos a entrenar los pipelines generados en los pasos anteriores. Para esto, debe realizar las siguientes tareas:

1. Separe los datos de entrenamiento en un conjunto de entrenamiento y de prueba  (la proporción queda a su juicio). En este paso, seleccione los ejemplos de forma aleatoria e intente mantener la distribución original de labels de cada clase en los conjuntos de prueba/entrenamiento. (vea la documentación de `train_test_split`). [1 puntos]


2. Defina un pipeline llamado `pipeline_xgboost` y otro llamado `pipeline_lightgbm`. Estos pipelines deben tener el mismo ColumnTransformer definido en la sección de preprocesamiento, pero deben variar los clasificadores de acuerdo al nombre de cada pipeline. [1 puntos]

3. Entrene los pipelines. [1 punto]



**Respuesta:**

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

# Se separan las variables de las clases
X = df.drop("label", axis=1)
y = df["label"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y) # Se crean los sets de entrenamiento y prueba considerando un 20% de los datos para prueba y manteniendo la proporción de clases


pipeline_xgboost = Pipeline([("transform", col_transformer),
                             ("classifier", XGBClassifier())]) # Se crea la pipeline para el XGBClassifier

pipeline_lightgbm = Pipeline([("transform", col_transformer),
                              ("classifier", LGBMClassifier())]) # Se crea la pipeline para le LGBMClassifier

# Se entrenan los pipelines
pipeline_xgboost.fit(X_train, y_train)
pipeline_lightgbm.fit(X_train, y_train)

[LightGBM] [Info] Number of positive: 860, number of negative: 13210
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.010440 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2530
[LightGBM] [Info] Number of data points in the train set: 14070, number of used features: 70
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.061123 -> initscore=-2.731797
[LightGBM] [Info] Start training from score -2.731797


Pipeline(steps=[('transform',
                 ColumnTransformer(transformers=[('categorical',
                                                  OneHotEncoder(),
                                                  ['Club_Position',
                                                   'Preffered_Foot',
                                                   'Work_Rate']),
                                                 ('numerical_standard',
                                                  StandardScaler(),
                                                  ['Height', 'Weight', 'Age',
                                                   'Ball_Control', 'Dribbling',
                                                   'Marking', 'Sliding_Tackle',
                                                   'Standing_Tackle',
                                                   'Aggression', 'Reactions',
                                                   'Interceptions', 'Vision',
                                                   'Composure', 'Crossing',
                                                   'Short_Pass', 'Long_Pass',
                                                   'Acceleration', 'Speed',
                                                   'Stamina', 'Strength',
                                                   'Balance', 'Agility',
                                                   'Jumping', 'Heading',
                                                   'Shot_Power', 'Finishing',
                                                   'Long_Shots', 'Curve',
                                                   'Freekick_Accuracy',
                                                   'Penalties', ...]),
                                                 ('numerical_minmax',
                                                  MinMaxScaler(),
                                                  ['Weak_foot',
                                                   'Skill_Moves'])])),
                ('classifier', LGBMClassifier())])

### 1.3 Resultados [6 puntos]

1. Calcule las métricas accuracy, precisión y recall de la clase positiva (la que indica que un jugador es seleccionado nacional) para evaluar el rendimiento de los distintos modelos. Verifique sus resultados usando `classification_report`. [2 puntos]

2. Explique qué implican los valores de accuracy, precisión y recall de la clase positiva (la que indica que un jugador es seleccionado nacional) y cómo influye la cantidad de ejemplos por clase en los resultados obtenidos. [2 puntos]

3. Explique qué métrica le parece más adecuada y concluya qué modelo tiene un mejor desempeño. [2 puntos]

**Respuesta:**

In [6]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, classification_report

# Se realizan las predicciones para ambos modelos con el set de prueba
pred_xgboost = pipeline_xgboost.predict(X_test)
pred_lightgbm = pipeline_lightgbm.predict(X_test)

# Se calculan el accuracy, precision y recall del modelo xgboost
accuracy_xgboost = accuracy_score(y_test, pred_xgboost)
precision_xgboost = precision_score(y_test, pred_xgboost)
recall_xgboost = recall_score(y_test, pred_xgboost)

# Se calculan el accuracy, precision y recall del modelo lightgbm
accuracy_lightgbm = accuracy_score(y_test, pred_lightgbm)
precision_lightgbm = precision_score(y_test, pred_lightgbm)
recall_lightgbm = recall_score(y_test, pred_lightgbm)

# Se realizan los classification report para ambos modelos
report_xgboost = classification_report(y_test, pred_xgboost)
report_lightgbm = classification_report(y_test, pred_lightgbm)

print("Resultados para modelo xgboost:")
print(f'Accuracy: {accuracy_xgboost}')
print(f'Precision: {precision_xgboost}')
print(f'Recall: {recall_xgboost} \n')

print("Resultados para modelo lightgbm:")
print(f'Accuracy: {accuracy_lightgbm}')
print(f'Precision: {precision_lightgbm}')
print(f'Recall: {recall_lightgbm} \n')

print("Classification report para ambos modelos:")
print(f'Xgboost: \n {report_xgboost}')
print(f'Lightgbm: \n {report_lightgbm}')

Resultados para modelo xgboost:
Accuracy: 0.9374644684479818
Precision: 0.46153846153846156
Recall: 0.13953488372093023 

Resultados para modelo lightgbm:
Accuracy: 0.9377487208641273
Precision: 0.4583333333333333
Recall: 0.10232558139534884 

Classification report para ambos modelos:
Xgboost: 
               precision    recall  f1-score   support

           0       0.95      0.99      0.97      3303
           1       0.46      0.14      0.21       215

    accuracy                           0.94      3518
   macro avg       0.70      0.56      0.59      3518
weighted avg       0.92      0.94      0.92      3518

Lightgbm: 
               precision    recall  f1-score   support

           0       0.94      0.99      0.97      3303
           1       0.46      0.10      0.17       215

    accuracy                           0.94      3518
   macro avg       0.70      0.55      0.57      3518
weighted avg       0.91      0.94      0.92      3518



2.- El accuracy indica que en general hubo un buen desempeño en el total de predicciones realizadas por los modelos, ya que indica que aproximadamente un 94% de los valores predichos fueron correctos. En lo que respecta a la presición, esta nos indica que un 50% aproximado de las clases predichas como positivas son realmente positivas, lo que es similar a lanzar una moneda. En el caso del recall se tiene un resultado bastante malo, ya que lo que dice es que se predice un gran número de falsos negativos, es decir, la gran mayoría de las clases que realmente son positivas son predichas como negativas (un 15% aproximado en este caso). Esto claramente puede ser un problema ocasionado por la disparidad de datos, ya que por lo que se puede notar las predicciones de la clase 0 son mucho mejores que las de la clase 1, que tiene menos datos.

3.- Para este caso y considerando el desbalance de los datos, la métrica que es más útil es la de recall, ya que de este modo se hace notar aún más el desbalance de los datos y el mal funcionamiento del modelo. En caso contrario, el accuracy en este caso es una métrica bastante mala, ya que al estar desbalanceados los datos se tiene una métrica muy buena cuando eso no se refleja en el funcionamiento clasificatorio del modelo.
En cuanto a cual modelo es mejor, se podría decir que sería el que tuviese la mejor métrica de recall como se dijo anteriormente, por lo que sería el modelo de Xgboost el mejor aunque sigue siendo un resultado bastante malo el que entregó.

## 2. Predicción de posiciones de jugadores [4 puntos]

<p align="center">
  <img src="https://pbs.twimg.com/media/E1rfA1aWEAYU6Ny.jpg" width="300">
</p>

En una nueva jornada de desmesuradas transacciones deportivas, Renacín escuchó a sus colegas discutir acerca de que el precio de cada jugador depende en gran medida de la posición en la cancha en la que juega. Y además, que hay bastantes jugadores nuevos que no tienen muy claro en que posición verdaderamente brillarían, por lo que actualmente puede que actualmente estén jugando en posiciones sub-optimas.

Viendo que los resultados del primer análisis no son tan esperanzadores, el corporeo los comanda a cambiar su tarea: ahora, les solicita que construyan un clasificador enfocado en predecir la mejor posición de los jugadores en la cancha según sus características.

Para lograr esto, primero, les pide que etiqueten de la siguiente manera los valores que aparecen en el atributo `Club_Position`, pidiendo que agrupen los valores en los siguientes grupos:

**Nota**:  Renacín les recalca que **no deben utilizar los valores ```Sub``` y ```Res``` de esta columna**.

```python
ataque = ['ST', 'CF']
central_ataque = ['RW', 'CAM', 'LW']
central = ['RM', 'CM', 'LM']
central_defensa = ['RWB', 'CDM', 'LWB']
defensa = ['RB', 'CB', 'LB']
arquero = ['GK']
```

La elección del clasificador se justificar en base a la siguiente [guía](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html) y se deben comentar los resultados obtenidos en la clasificación.

**Tareas:** [1 punto por tarea]

1. Aplique las etiquetas descritas anteriormente en cada uno de los valores señalados en esta sección.
2. Cuente cuántos por clase quedan.
3. Entrene el nuevo pipeline y ejecute una evaluación de este.  
4. Comente los resultados obtenidos.

**Respuesta:**

In [7]:
# Se crea un diccionario para mapear los nuevos valores de "Club_Position"
mapa = {
    "ST": "ataque", "CF": "ataque",
    "RW": "central_ataque", "CAM": "central_ataque", "LW": "central_ataque",
    "RM": "central", "CM": "central", "LM": "central",
    "RWB": "central_defensa", "CDM": "central_defensa", "LWB": "central_defensa",
    "RB": "defensa", "CB": "defensa", "LB": "defensa",
    "GK": "arquero"
}

df["Club_Position"] = df["Club_Position"].map(mapa) # Se realiza el mapeo

conteo = df['Club_Position'].value_counts() # Se realiza un conteo de cada clase en "Club_Position"
print(conteo)

Club_Position
defensa            1180
central             907
arquero             632
central_ataque      581
ataque              430
central_defensa     209
Name: count, dtype: int64


In [8]:
from sklearn.svm import LinearSVC

df = df.dropna(subset=["Club_Position"]) # Se eliminan los valores NaN de "Club_Position"

# Se dividen los datos de la clase a predecir
X = df.drop(["Club_Position"], axis=1)
y = df["Club_Position"]

# Se crean los conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

categorical_data = ["Preffered_Foot", "Work_Rate"] # Se crea una lista con las variables categoricas, eliminando las que no aportan información como el nombre, nacionalidad y national position. Además se elimina el objetivo a predecir "Club_Position"
numerical_data_standard = ["Height", "Weight", "Age", "Ball_Control", "Dribbling", "Marking", "Sliding_Tackle", "Standing_Tackle", "Aggression", "Reactions", "Interceptions", "Vision", "Composure", "Crossing", "Short_Pass", "Long_Pass", "Acceleration", "Speed", "Stamina", "Strength", "Balance", "Agility", "Jumping", "Heading", "Shot_Power", "Finishing", "Long_Shots", "Curve", "Freekick_Accuracy", "Penalties", "Volleys"] # Se crea una lista con las variables numéricas con las que se usará la transformación StandardScaler
numerical_data_minmax = ["Weak_foot", "Skill_Moves"] # Se crea una lista con las variables numéricas con las que se usará la transformación MinMaxScaler

col_transformer = ColumnTransformer(
    transformers=[("categorical", OneHotEncoder(), categorical_data),
                  ("numerical_standard", StandardScaler(), numerical_data_standard),
                  ("numerical_minmax", MinMaxScaler(), numerical_data_minmax)]) # Se crea el ColumnTransformer con las transformaciones esperadas para cada conjunto de variables

# Se crea la pipeline usando el columnTransformer creado anteriormente y el modelo LinearSVC, el cual fue elegido a partir de la guía entregada, ya que se trata de un modelo de clasificación con menos de 100k datos
pipeline_linearsvc = Pipeline([("transform", col_transformer),
                              ("classifier", LinearSVC())])

# Se entrena el modelo
pipeline_linearsvc.fit(X_train, y_train)

# Se realizan las predicciones
pred_linearsvc = pipeline_linearsvc.predict(X_test)

# Se hace un classification report con los datos predecidos
report_linearsvc = classification_report(y_test, pred_linearsvc)

print("Classification report para modelo LinearSVC:")
print(report_linearsvc)

/usr/local/lib/python3.10/dist-packages/sklearn/svm/_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


Classification report para modelo LinearSVC:
                 precision    recall  f1-score   support

        arquero       1.00      1.00      1.00       126
         ataque       0.75      0.86      0.80        86
        central       0.63      0.70      0.66       182
 central_ataque       0.57      0.30      0.40       116
central_defensa       0.50      0.14      0.22        42
        defensa       0.80      0.98      0.88       236

       accuracy                           0.76       788
      macro avg       0.71      0.66      0.66       788
   weighted avg       0.74      0.76      0.73       788



/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Se puede notar que los resultados obtenidos no son muy buenos en general, aunque el modelo si logra clasificar de muy buena forma a los jugadores que tienen la posición de "arquero", así como a los jugadores con posición "ataque" y "defensa", aunque con peores resultados para estos últimos. Otra cosa destacable de estos resultados son los malos resultados que se tienen para todos los jugadores que estén ligados a una posición de centro como "central", "central_ataque" y "central_defensa". Una posible causa de los malos resultados es que estas últimas dos clases son las que menos datos tienen en el dataset, aunque también es posible que el modelo falle debido a que estas clases pueden tener una gran similitud entre ellas sin atributos en los que se puedan diferenciar de manera clara, ya que en el caso de "ataque" también se cuenta con pocos datos, pero los resultados obtenidos son decentes.  

## 3. Predicciones de Seleccionados Nacionales para el Jere Klein [30 puntos]

<center>
<img src='https://www.radioactiva.cl/wp-content/uploads/2024/04/Jere-Klein-1-768x432.webp' width=500 />

Después de alcanzar la fama como cantante urbano, Jere Klein decide explorar una nueva faceta. Con su amor por el fútbol y convencido de que los artistas urbanos poseen un talento y versatilidad excepcionales, Jere se embarca en un proyecto innovador: desarrollar un sistema de inteligencia artificial capaz de identificar a jugadores que tienen potencial para convertirse en futbolistas profesionales. Su teoría es que muchos artistas del género urbano chileno, con sus habilidades únicas y su disciplina, podrían destacarse también en el deporte. Con este sistema, Jere espera no solo abrir nuevas oportunidades para sus colegas artistas, sino también demostrar la amplia gama de talentos que pueden ofrecer.

### 2.1 ¿Qué modelo de árbol es más de "pana"? [10 puntos]

<center>
<img src='https://64.media.tumblr.com/39189215a7d3d96823cb359f35b44e05/tumblr_psmrhrR3Xw1qf5hjqo4_540.gif' width=300 />


**Tareas**


1. Considerando el la variable llamada `label` creada en la sección 1.1. Para determinar cuál modelo de árbol sería más adecuado para la tarea en cuestión, utilice PyCaret. Este deberá centrarse exclusivamente en modelos de tipo árbol. Jere ha especificado que busca un modelo que tome decisiones rápidamente y que tenga una baja tasa de falsos positivos, ya que planea invertir en estos jugadores. [3 puntos] Para la comparación, utilice los siguientes modelos:

```python
['et', 'rf', 'dt', 'xgboost', 'lightgbm', 'catboost']
```

3. Explique en brevemente que son los modelos de la siguiente lista `['et', 'rf', 'dt']` y como funcionan. [3 punto]

4. Tras realizar la comparación de modelos, seleccione aquel que muestre el mejor rendimiento en términos de velocidad y precisión, especialmente en la reducción de falsos positivos. Utilice la función `evaluate_model` de PyCaret para revisar y analizar los resultados obtenidos en los siguientes aspectos:

  - **Confusión Matrix**: ¿Cómo se encuentran la tasa de verdaderos positivos y verdaderos negativos?
  - **Threshold**: ¿Es acaso el umbral por defecto del modelo el mejor para las predicciones?
  - **Feature Importance**: ¿Cuáles son las variables con mejor desempeño? ¿A qué podría deberse esto?
  - **Learning Curve**: ¿El modelo presenta algún problema?

  [4 puntos]

3.- El modelo 'et' es Extra Trees (ET), que corresponde a un algoritmo de ensamble que se basa en árboles de precisión. Lo que lo diferencia de los árboles estándar y Rnadom Forest, es que ET agrega aleatoriedad adicional a la construcción de los árboles, ya que en cada nodo de división, en vez de buscar el mejor umbral posible para dividir, ET selecciona aleatoriamente varios umbrales y elige el mejor entre ellos. Lo anterior hace que ET sea más rápido de entrenar que Random Forest, pero puede que necesite más árboles para alcanzar el mismo nivel de precisión.

En cuanto a 'rf', este corresponde a Random Forest (RF), que al igual que ET es un algoritmo de ensamble basado en árboles de decisión. Este algoritmo construye múltiples árboles durante el entrenamiento y los combina para obtener predicciones más precisas. Los árboles son construidos en paralelo, lo que hace que RF sea adecuado para conjuntos de datos grandes y complejos. RF también introduce aleatoriedad al considerar un subconjunto aleatorio de características en cada nodo de división, lo que ayuda a reducir el sobreajuste y mejorar la generalización.

Por último, 'dt' corresponde a Decision Tree (DT), que es un modelo de aprendizaje supervisado para clasificación y regresión.  Divide el conjunto de datos en subconjuntos más pequeños basados en características específicas, de modo que las decisiones se tomen de manera secuencial en función de estas divisiones. El objetivo es dividir el conjunto de datos de manera que las clases sean lo más homogéneas posible en los nodos hoja.

4.- Luego de la prueba de los modelos, se concluye que el mejor es ET, pues es el que obtiene valores más altos en precision, recall y F1 score, que son métricas relevantes al reducir los falsos positivos. Particularmente precision es importante, ya que se calcula como el número de verdaderos positivos dividido por la suma de verdaderos positivos y falsos positivos.

En cuanto a la velocidad, ET es el que más demora, pero considerando que los tiempos de los distintos modelos no presentan diferencias relevantes para este ejemplo, la elección se guía principalmente por las métricas.

Pasando al análisis de los resultados tenemos lo siguiente:

  - **Confusión Matrix**: De acuerdo a la matriz de confusión hay 57 verdaderos positivos, 1071 verdaderos negativos, 53 falsos negativos y 1 falso positivo. Es decir, efectivamente los falsos positivos son muy pocos, no obstante, de todos los positivos (reales), casi la mitad se clasifican como negativos. Esto tiene relación con que las clases están desbalanceadas, pues hay más negativos que positivos.
  - **Threshold**: De acuerdo a lo observado en el gráfico, se puede decir que el umbral por defecto del modelo es el mejor, ya que maximiza precision, recall y f1 score.
  - **Feature Importance**: El top de 4 variables con mejor desempeño es: 'Name', 'National_Position_Sub', 'National_Position_GK' y 'Nationality', donde 'Name' tiene una importancia particularmente alta. En el caso de 'Name', dado que es una variable categórica que tiene prácticamente un valor distinto por fila, corresponde a un caso que hay que analizar con detención. Si bien un problema como este podría darse por múltiples factores, existe la probabilidad de que existe overfitting y que el modelo le esté dando demasiada importancia a características irrelevantes. En los casos de 'National_Position_Sub' y 'National_Position_GK', que hacen referencia a si el jugador ocupa 2 posiciones (Sub o suplente, y GK o arquero) en su país, lo cual puede tener gran relevancia al ser indicadores sobre la habilidad y el estilo de juego de cada jugador. Por su parte, 'Nationality' puede tener relación con lo que se quiere predecir (si el jugador es seleccionado, o bien, si es bueno) ya que el nivel de desarrollo de un deporte varía de un país a otro.
  - **Learning Curve**: De acuerdo al gráfico de Learning Curve podría haber un problema de overfitting, ya que en una zona del gráfico la curva para el cross validation score baja, mientras la del training score de mantiene.


**Respuesta**

In [ ]:
!pip install pycaret

In [ ]:
pip install catboost

In [9]:
from pycaret.classification import *
clf1 = setup(data=df, target='label', session_id=123)

,Description,Value
0,Session id,123
1,Target,label
2,Target type,Binary
3,Original data shape,"(3939, 40)"
4,Transformed data shape,"(3939, 76)"
5,Transformed train set shape,"(2757, 76)"
6,Transformed test set shape,"(1182, 76)"
7,Numeric features,33
8,Categorical features,6
9,Rows with missing values,90.7%


In [10]:
# et es el mejor
best_model = compare_models(include = ['et', 'rf', 'dt', 'xgboost', 'lightgbm', 'catboost'], sort='F1')

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
et,Extra Trees Classifier,0.9485,0.8602,0.4494,0.9889,0.6143,0.5913,0.6460,1.2690
rf,Random Forest Classifier,0.9347,0.8796,0.3014,0.9750,0.4326,0.4127,0.4984,0.5590
dt,Decision Tree Classifier,0.9068,0.4998,0.0000,0.0000,0.0000,-0.0007,-0.0019,0.2500
xgboost,Extreme Gradient Boosting,0.9068,0.7884,0.0000,0.0000,0.0000,-0.0007,-0.0019,0.4970
lightgbm,Light Gradient Boosting Machine,0.9068,0.6873,0.0000,0.0000,0.0000,-0.0007,-0.0019,0.7850
catboost,CatBoost Classifier,0.9068,0.7798,0.0000,0.0000,0.0000,-0.0007,-0.0019,8.2680


Processing:   0%|          | 0/29 [00:00<?, ?it/s]

In [14]:
et_model = create_model('et')
evaluate_model(et_model)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9565,0.8966,0.5200,1.0000,0.6842,0.6633,0.7045
1,0.9420,0.8381,0.3846,1.0000,0.5556,0.5310,0.6012
2,0.9457,0.9091,0.4231,1.0000,0.5946,0.5705,0.6318
3,0.9457,0.8844,0.4231,1.0000,0.5946,0.5705,0.6318
4,0.9565,0.9149,0.5385,1.0000,0.7000,0.6788,0.7168
5,0.9601,0.8623,0.5769,1.0000,0.7317,0.7118,0.7434
6,0.9312,0.8157,0.3077,0.8889,0.4571,0.4295,0.4995
7,0.9418,0.7170,0.3600,1.0000,0.5294,0.5056,0.5817
8,0.9491,0.8674,0.4400,1.0000,0.6111,0.5882,0.6455


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

### 2.2 Reducción de dimensionalidad [14 puntos]

<center>
<img src='https://i.kym-cdn.com/photos/images/original/002/258/560/668.gif' width=400 />

A pesar de los resultados obtenidos previamente, el manager de Jere ha solicitado el entrenamiento de un modelo de XGBoost utilizando los datos disponibles. Además, se debe proceder a realizar una reducción de dimensionalidad basada en la importancia de las características.

Para llevar a cabo esta tarea:

1. Inicie entrenando un modelo XGBoost con todas las características disponibles. [2 puntos]

2. Una vez el modelo esté entrenado, evalúe y clasifique las características según su importancia de forma descendente. [2 puntos]

3. Utilice esta clasificación para ejecutar una búsqueda recursiva de eliminación de características, eliminando progresivamente las menos importantes y evaluando el impacto en el desempeño del modelo hasta identificar las N características más críticas. [2 puntos]

4. Con este conjunto reducido de características, entrene un nuevo modelo y evalúe su rendimiento. [2 puntos]

5. Posteriormente, responda a las siguientes preguntas para una comprensión más profunda de los cambios y beneficios:

  - ¿El rendimiento del modelo con las características seleccionadas es similar al del modelo original? ¿Cómo se comparan en términos de precisión y robustez? [2 puntos]
  - ¿Cuáles son los beneficios potenciales de eliminar variables del modelo? Considere factores como la simplificación del modelo, reducción del tiempo de entrenamiento, y mejora en la capacidad de generalización. [2 puntos]
  - Comente si el modelo con menor dimensionalidad es más sencillo de explicar. Explique brevemente por qué la eliminación de ciertas características puede facilitar la comprensión y la explicación del comportamiento del modelo. [2 puntos]

Notar que con esta metodologia buscamos encontrar un punto entermedio entre número de festures y desempeño. por esto, si observa que al aumentar festires el aumento es despreciable, puede no considerar agregar más features a su modelo.

**Respuesta**

### 2.3 Calibración Probabilistica [6 puntos]

<center>
<img src='https://media2.giphy.com/media/l2Je4Ku0Cx292KWv6/200w.gif?cid=6c09b952y0sihtq9tb6sz8j2023x3zxxp3qx1ocgonkpkblj&ep=v1_gifs_search&rid=200w.gif&ct=g' width=400 />

Para lograr modelos más modulares, se recomienda realizar una calibración del modelo entrenado anteriormente, con el objetivo de obtener salidas que reflejen mayor modularidad.

1. Se solicita que utilice un método de calibración que asegure que las probabilidades generadas incrementen de manera monótona. Una métrica ampliamente utilizada para evaluar la precisión de la calibración de un modelo es el Brier Score. Calcule el Brier Score para el modelo tanto antes como después de la calibración. Esto le permitirá realizar una comparación cuantitativa y determinar si la calibración ha mejorado el rendimiento del modelo. Para más información sobre el Brier Score, puede consultar el siguiente enlace: [Scikit-Learn - Brier Score Loss](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.brier_score_loss.html). [3 puntos]

2. Tras la calibración, examine y comente los resultados obtenidos. A su análisis añada una comparación visual de las ideales versus las salidas del modelo original (sin calibrar) y del modelo calibrado. [3 puntos]

**Respuesta**